## Scaleway Object Storage - Testing

https://www.simplecto.com/using-django-and-boto3-with-scaleway-object-storage/

* `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` can be obtained from the [credentials control panel](https://console.scaleway.com/project/credentials) under API Keys.
* `AWS_STORAGE_BUCKET_NAME` is the name of the bucket you create on [objects administration page](https://console.scaleway.com/object-storage/buckets)
* `AWS_DEFAULT_ACL` is set to public-read so that the objects can be pulled from a URL without any access keys or time-limited signatures.
* `AWS_S3_REGION_NAME` and `AWS_S3_ENDPOINT_URL` should be configured so that `boto3` knows to point to the Scaleway resources.

All of these are references in the Scaleways docs on Object Storage.

### Resources:
* https://github.com/liormizr/s3path/blob/master/docs/advance.rst
* https://www.scaleway.com/en/docs/object-storage-feature/
* https://www.scaleway.com/en/docs/how-to-migrate-object-storage-buckets-with-rclone/
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html

In [1]:
import tomli
from pathlib import Path
import boto3

In [9]:
boto3.__version__

'1.17.107'

In [2]:
APP_CONFIG_FILE = 'app_config.toml'

In [3]:
with open(Path('..')/APP_CONFIG_FILE, encoding="utf-8") as f:
    app_config = tomli.load(f)

In [4]:
AWS_ACCESS_KEY_ID = app_config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = app_config['AWS']['AWS_SECRET_ACCESS_KEY']
AWS_STORAGE_BUCKET_NAME = app_config['AWS']['AWS_STORAGE_BUCKET_NAME']
AWS_DEFAULT_ACL = app_config['AWS']['AWS_DEFAULT_ACL']
AWS_S3_REGION_NAME = app_config['AWS']['AWS_S3_REGION_NAME']
AWS_S3_ENDPOINT_URL =  app_config['AWS']['AWS_S3_ENDPOINT_URL']

In [5]:
AWS_S3_ENDPOINT_URL

'https://s3.fr-par.scw.cloud'

In [6]:
s3 = boto3.client('s3', 
        region_name=AWS_S3_REGION_NAME, 
        endpoint_url=AWS_S3_ENDPOINT_URL, 
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

In [14]:
def download_s3_folder(s3, bucket_name, s3_folder, local_dir=None):
    filecount = 0
    files = []
    if not local_dir.exists():
        Path.mkdir(local_dir)
    print('Writing files to local dir: ' + Path(local_dir).as_posix())
    print(bucket_name)
    bucket_list=s3.list_objects(Bucket=bucket_name)['Contents']
    for s3_key in bucket_list:
        s3_object = s3_key['Key']
        if not s3_object.endswith("/"):
            filepath = Path(local_dir)/s3_object
            print(filepath.as_posix())
            s3.download_file(bucket_name, s3_object, filepath.as_posix())
            filecount+=1
            files.append(s3_object)
        else:
            if not (Path(local_dir)/s3_object).exists():
                print('Creating dir: ' + (Path(local_dir)/s3_object).as_posix())
                Path.mkdir(Path(local_dir)/s3_object)
    return filecount, files

In [23]:
def list_s3_folder(s3, bucket_name, s3_folder):
    return [s3_key['Key'] for s3_key in s3.list_objects(Bucket=bucket_name)['Contents'] if s3_key['Key'].endswith("/")]

In [25]:
list_s3_folder(s3, AWS_STORAGE_BUCKET_NAME, 'data/FIT')

['data/']

In [26]:
filecount, files = download_s3_folder(s3, AWS_STORAGE_BUCKET_NAME, 'data/FIT', Path.home()/'tmp/test-scaleway')

Writing files to local dir: /Users/mjboothaus/tmp/test-scaleway
emmaus-walking
/Users/mjboothaus/tmp/test-scaleway/data/FIT/B2M/2020-07-13-085452-Walking-Michael and Ai Leen’s Apple Watch.fit


FileNotFoundError: [Errno 2] No such file or directory: '/Users/mjboothaus/tmp/test-scaleway/data/FIT/B2M/2020-07-13-085452-Walking-Michael and Ai Leen’s Apple\xa0Watch.fit.4CfeCC67'

In [13]:
print(str(filecount) + ' files downloaded')
print(files)

NameError: name 'filecount' is not defined

## Alternate approach - `s3path` library

In [14]:
import boto3
from s3path import S3Path

In [15]:
s3 = boto3.client('s3', 
        region_name=AWS_S3_REGION_NAME, 
        endpoint_url=AWS_S3_ENDPOINT_URL, 
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

In [19]:
boto3.setup_default_session(
        region_name=AWS_S3_REGION_NAME, 
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

In [34]:
bucket_path = S3Path('/emmaus-walking/')

In [37]:
bucket_path

S3Path('/emmaus-walking')

In [23]:
scaleway_resource = boto3.resource('s3',
        endpoint_url=AWS_S3_ENDPOINT_URL, 
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name=AWS_S3_REGION_NAME)

In [32]:
AWS_S3_REGION_NAME

'fr-par'

In [28]:
from s3path import PureS3Path, register_configuration_parameter
#[path for path in bucket_path.iterdir() if path.is_dir()]

In [30]:
scaleway_bucket_path = PureS3Path('/emmaus-walking/')

In [26]:
register_configuration_parameter(scaleway_bucket_path, resource=scaleway_resource)

In [33]:
scaleway_bucket_path.parts

('/', 'emmaus-walking')

In [35]:
def connect_s3_session_scaleway(bucket_name):
        
    session = boto3.session.Session()

    s3_client = session.client(
        service_name='s3',
        region_name=AWS_S3_REGION_NAME,
        use_ssl=True,
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        endpoint_url=AWS_S3_ENDPOINT_URL,
    )  

    return s3_client


In [36]:
s3_client = connect_s3_session_scaleway('emmaus-walking')

In [37]:
s3_client

AttributeError: 'S3' object has no attribute 'download_s3_folder'